In [1]:
from codeocean import CodeOcean
from codeocean.data_asset import DataAssetSearchParams, DataAssetAttachParams
from codeocean.components import SearchFilter
import os
from aind_dynamic_foraging_data_utils.code_ocean_utils import get_assets, attach_data, add_data_asset_path 
import re
from datetime import datetime

import pandas as pd
import numpy as np

from pathlib import Path
import matplotlib.pyplot as plt
import json
from aind_dynamic_foraging_behavior_video_analysis.kinematics.tongue_kinematics_utils import get_session_name_from_path, plot_keypoint_confidence_analysis
import glob
import yaml


In [ ]:
# # example

# client = CodeOcean(domain="https://codeocean.allenneuraldynamics.org",
#                    token=os.getenv("API_SECRET"))

# needle = "behavior_751181_2025-02-27"

# # A) Free-text query (supports patterns like name:... per docs)
# params_query = DataAssetSearchParams(
#     offset=0, limit=100,
#     sort_order="desc", sort_field="name",
#     archived=False, favorite=False,
#     query=f"name:{needle}"
# )
# res_query = client.data_assets.search_data_assets(params_query)
# print("Query matches:", len(res_query.results))
# for r in res_query.results:
#     print(r.id, r.name)


# # --- helper: pick most recent videoprocessed asset from a list of asset models ---
# def most_recent_videoprocessed_asset(assets):
#     """
#     assets: iterable of DataAsset model objects with .name and .id
#     Returns: (asset, parsed_datetime) or (None, None) if none match
#     """
#     pat = re.compile(r"_videoprocessed_(\d{4}-\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})")
#     best = None
#     best_dt = None
#     for a in assets:
#         m = pat.search(a.name)
#         if not m:
#             continue
#         dt = datetime.strptime(f"{m.group(1)}_{m.group(2)}", "%Y-%m-%d_%H-%M-%S")
#         if (best_dt is None) or (dt > best_dt):
#             best_dt = dt
#             best = a
#     return best, best_dt

# # Filter the searched assets down to the ones with _videoprocessed_ timestamps
# videoprocessed_assets = [r for r in res_query.results if "_videoprocessed_" in r.name]

# latest_asset, latest_dt = most_recent_videoprocessed_asset(videoprocessed_assets)

# if latest_asset is None:
#     print("No videoprocessed assets found in search results.")
# else:
#     # Choose a clear mount name (adjust to your convention if needed)
#     # ex: "behavior_751181_2025-02-27_videoprocessed"
#     # mount_name = f"{needle}_videoprocessed"
#     mount_name = latest_asset.mount

#     # Attach
#     attach_params = [DataAssetAttachParams(id=latest_asset.id, mount=mount_name)]
#     results = client.capsules.attach_data_assets(
#         capsule_id=os.getenv("CO_CAPSULE_ID"),
#         attach_params=attach_params,
#     )

#     print(f"Attached most recent videoprocessed asset:")
#     print(f"  ID:   {latest_asset.id}")
#     print(f"  Name: {latest_asset.name}")
#     print(f"  Time: {latest_dt.isoformat()}")
    





Query matches: 11
d330c095-44b6-43bf-bf8e-eaf435bea9e9 behavior_751181_2025-02-27_11-24-47_sorted_curated_2025-03-25_22-37-19
9b285728-1647-49d7-83d7-309892cf1160 behavior_751181_2025-02-27_11-24-44_videoprocessed_2025-10-29_21-59-06
ae5f98b1-f883-4615-825b-cfeb6a2ed43b behavior_751181_2025-02-27_11-24-44_videoprocessed_2025-10-24_21-43-19
4f5c31bc-25fa-43ea-97d2-7ed63dad3b00 behavior_751181_2025-02-27_11-24-44_videoprocessed_2025-10-24_21-43-19
2e3d9361-3347-48a4-9fac-e0180450db4b behavior_751181_2025-02-27_11-24-44_videoprocessed_2025-10-24_21-43-19
ad853a27-d3fe-41fe-9d0b-ec26a418b329 behavior_751181_2025-02-27_11-24-44_videoprocessed_2025-09-09_18-59-42
c5fbf157-805e-4d29-ba89-83a053545e22 behavior_751181_2025-02-27_11-24-44_sorted_2025-03-11_00-30-26
ad291c3a-bf5e-4d03-af52-e59d0b183538 behavior_751181_2025-02-27_11-24-44_sorted-opto-bp_2025-03-21_01-06-52
a701b4e0-9560-4e45-85b2-6b12187c0ab9 behavior_751181_2025-02-27_11-24-44_curated-ZhixiaoSu_2025-03-25_18-32-11
12199fde-530b-4

In [4]:

# Base directory
data_root = Path("/root/capsule/data")

# List of session names you provided (unchanged)
# List of session names you provided (unchanged)
session_names = [
    "behavior_716325_2024-05-31_10-31-14",
    "behavior_717121_2024-06-15_10-00-58",
    "behavior_717259_2024-06-28_11-17-19",
    "behavior_717263_2024-07-24_10-40-05",
    "behavior_751004_2024-12-20_13-26-11",
    "behavior_751004_2024-12-21_13-28-28",
    "behavior_751004_2024-12-22_13-09-17",
    "behavior_751004_2024-12-23_14-20-03",
    "behavior_751769_2025-01-16_11-32-05",
    "behavior_751769_2025-01-17_11-37-39",
    "behavior_751769_2025-01-18_10-15-25",
    "behavior_758017_2025-02-04_11-57-38",
    "behavior_758017_2025-02-05_11-42-34",
    "behavior_758017_2025-02-06_11-26-14",
    "behavior_758017_2025-02-07_14-11-08",
    "behavior_751766_2025-02-11_11-53-38",
    "behavior_751766_2025-02-13_11-31-21",
    "behavior_751766_2025-02-14_11-37-11",
    "behavior_751766_2025-02-15_12-08-11",
    "behavior_751181_2025-02-25_12-12-35",
    "behavior_751181_2025-02-26_11-51-19",
    "behavior_751181_2025-02-27_11-24-47",
    "behavior_754897_2025-03-11_12-07-41",
    "behavior_754897_2025-03-12_12-23-15",
    "behavior_754897_2025-03-13_11-20-42",
    "behavior_754897_2025-03-14_11-28-53",
    "behavior_754897_2025-03-15_11-32-18",
    "behavior_758018_2025-03-19_11-16-44",
    "behavior_758018_2025-03-20_11-53-05",
    "behavior_758018_2025-03-21_11-00-34",
    "behavior_752014_2025-03-25_12-09-20",
    "behavior_752014_2025-03-26_11-18-57",
    "behavior_752014_2025-03-27_12-03-59",
    "behavior_752014_2025-03-28_11-04-59",
    "behavior_761038_2025-04-15_10-25-11",
    "behavior_761038_2025-04-16_10-39-10",
    "behavior_761038_2025-04-17_11-03-16",
    "behavior_761038_2025-04-18_12-37-39",
    # "ecephys_763360_2025-04-15_12-16-29",
    # "ecephys_763360_2025-04-16_13-29-55",
    "behavior_782394_2025-04-22_10-53-28",
    "behavior_782394_2025-04-23_10-51-17",
    "behavior_782394_2025-04-24_12-07-34",
    "behavior_782394_2025-04-25_11-13-21",
    "behavior_763590_2025-05-01_10-59-18",
    "behavior_763590_2025-05-02_11-07-09",
    "behavior_781166_2025-05-13_14-04-27",
    "behavior_781166_2025-05-14_14-18-28",
    "behavior_781166_2025-05-15_14-20-51",
    
    # Newly added sessions:
    "behavior_784806_2025-06-17_14-59-23",
    "behavior_784806_2025-06-18_13-39-50",
    "behavior_784806_2025-06-20_13-39-16",
    "behavior_791691_2025-06-24_13-21-29",
    "behavior_791691_2025-06-25_14-06-10",
    "behavior_791691_2025-06-26_13-39-26",
    "behavior_791691_2025-06-27_13-54-30",
    "behavior_784803_2025-07-01_13-58-26",
    "behavior_784803_2025-07-02_13-41-41",
    "behavior_784803_2025-07-03_13-55-13",
]


pred_csv_list = []

# --- Code Ocean client ---
client = CodeOcean(
    domain="https://codeocean.allenneuraldynamics.org",
    token=os.getenv("API_SECRET"),
)

# Helper to parse the videoprocessed timestamp from asset.name
_vp_pat = re.compile(r"_videoprocessed_(\d{4}-\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})$")
_allowed_days = {"2025-10-28", "2025-10-29", "2025-11-07"}

def _parse_vp_dt(asset_name: str):
    m = _vp_pat.search(asset_name)
    if not m:
        return None, None
    day, hms = m.group(1), m.group(2)
    try:
        return day, datetime.strptime(f"{day}_{hms}", "%Y-%m-%d_%H-%M-%S")
    except Exception:
        return day, None

for session in session_names:
    # Extract base (everything up to YYYY-MM-DD)   [unchanged]
    match = re.match(r"^(.*\d{4}-\d{2}-\d{2})", session)
    if not match:
        print(f"⚠️ Could not parse session name: {session}")
        continue
    session_base = match.group(1)

    # --- Use CO API to find assets for this session_base ---
    try:
        params = DataAssetSearchParams(
            offset=0, limit=200, sort_order="desc", sort_field="name",
            archived=False, favorite=False, query=f"name:{session_base}"
        )
        res = client.data_assets.search_data_assets(params)
        # keep only assets that contain '_videoprocessed_' AND were processed on allowed days
        vp_assets = []
        for a in res.results:
            if "_videoprocessed_" not in a.name:
                continue
            day, dt = _parse_vp_dt(a.name)
            if day in _allowed_days and dt is not None:
                vp_assets.append((a, dt))
        if not vp_assets:
            print(f"⚠️ No match found for {session}")
            continue

        # pick most recent by parsed timestamp (second date after 'videoprocessed')
        latest_asset, _latest_dt = max(vp_assets, key=lambda x: x[1])

        # Attach using the asset's own mount name (as requested)
        attach = [DataAssetAttachParams(id=latest_asset.id, mount=latest_asset.mount)]
        _ = client.capsules.attach_data_assets(
            capsule_id=os.getenv("CO_CAPSULE_ID"),
            attach_params=attach,
        )

        # The folder for this session is the mounted path under data_root
        folder = data_root / latest_asset.mount

    except Exception as e:
        print(f"⚠️ CO API error for {session}: {e}")
        continue

    # --- look for predictions.csv under folder ---
    matches = list((folder / "pred_outputs" / "video_preds").glob("*predictions.csv"))
    if not matches:
        print(f"⚠️ No predictions.csv files found for {session}")
    else:
        # If multiple matches, pick the most recent (sorted by name)  [unchanged]
        csv_path = sorted(matches)[-1]
        if csv_path.exists():
            pred_csv_list.append(str(csv_path))
        else:
            print(f"⚠️ Predictions CSV missing for {session}")

# Print and save results (unchanged)
print("pred_csv_list = [")
for path in pred_csv_list:
    print(f'    "{path}",')
print("]")

save_path = Path("/root/capsule/scratch/pred_csv_list_20250113.json")
save_path.parent.mkdir(parents=True, exist_ok=True)

with open(save_path, "w") as f:
    json.dump(pred_csv_list, f, indent=2)

print(f"Saved {len(pred_csv_list)} paths to {save_path}")


⚠️ No match found for behavior_717121_2024-06-15_10-00-58
⚠️ No predictions.csv files found for behavior_781166_2025-05-13_14-04-27
⚠️ No predictions.csv files found for behavior_781166_2025-05-14_14-18-28
⚠️ No predictions.csv files found for behavior_781166_2025-05-15_14-20-51
pred_csv_list = [
    "/root/capsule/data/behavior_716325_2024-05-31_10-31-14_videoprocessed_2025-10-28_23-21-23/pred_outputs/video_preds/video_predictions.csv",
    "/root/capsule/data/behavior_717259_2024-06-28_11-17-19_videoprocessed_2025-10-28_23-21-23/pred_outputs/video_preds/video_predictions.csv",
    "/root/capsule/data/behavior_717263_2024-07-24_10-40-05_videoprocessed_2025-10-28_23-21-23/pred_outputs/video_preds/video_predictions.csv",
    "/root/capsule/data/behavior_751004_2024-12-20_13-26-07_videoprocessed_2025-10-28_23-21-23/pred_outputs/video_preds/video_predictions.csv",
    "/root/capsule/data/behavior_751004_2024-12-23_14-19-57_videoprocessed_2025-10-28_23-21-23/pred_outputs/video_preds/video_

In [8]:
# Base directory
data_root = Path("/root/capsule/data")

# ... session_names and client setup omitted for brevity ...

# Helper to parse the videoprocessed timestamp from asset.name
_vp_pat = re.compile(r"_videoprocessed_(\d{4}-\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})$")
_allowed_days = {"2025-10-28", "2025-10-29", "2025-11-07"}

def _parse_vp_dt(asset_name: str):
    m = _vp_pat.search(asset_name)
    if not m:
        return None, None
    day, hms = m.group(1), m.group(2)
    try:
        return day, datetime.strptime(f"{day}_{hms}", "%Y-%m-%d_%H-%M-%S")
    except Exception:
        return day, None

for session in session_names:
    # Extract base (everything up to YYYY-MM-DD)   [unchanged]
    match = re.match(r"^(.*\d{4}-\d{2}-\d{2})", session)
    if not match:
        print(f"⚠️ Could not parse session name: {session}")
        continue
    session_base = match.group(1)

    # --- Ensure original/raw session attached (no suffixes) ---
    # raw name pattern: <session_base>_<HH-MM-SS>, with no extra suffix
    raw_pattern = re.compile(
        r"^" + re.escape(session_base) + r"_\d{2}-\d{2}-\d{2}$"
    )

    # 1) Check local /root/capsule/data for any matching raw folder
    try:
        local_raw_dirs = [
            p for p in data_root.iterdir()
            if p.is_dir() and raw_pattern.match(p.name)
        ]
    except FileNotFoundError:
        local_raw_dirs = []

    if local_raw_dirs:
        # Already attached; nothing to do
        # (optional) verbose:
        # print(f"✓ Raw session already present for base {session_base}: {local_raw_dirs[0].name}")
        pass
    else:
        # 2) Not present locally -> search CO for matching raw asset and attach
        try:
            params_raw = DataAssetSearchParams(
                offset=0,
                limit=100,
                sort_order="desc",
                sort_field="name",
                archived=False,
                favorite=False,
                query=f"name:{session_base}",
            )
            res_raw = client.data_assets.search_data_assets(params_raw)

            raw_assets = [
                a for a in res_raw.results
                if raw_pattern.match(a.name)
            ]

            if not raw_assets:
                print(f"⚠️ No raw session asset found for base {session_base}")
            else:
                # deterministic choice if more than one, though you expect exactly one
                raw_asset = sorted(raw_assets, key=lambda a: a.name)[0]

                # Attach using its own name as mount so folder matches asset.name
                attach_raw = [
                    DataAssetAttachParams(id=raw_asset.id, mount=raw_asset.name)
                ]
                client.capsules.attach_data_assets(
                    capsule_id=os.getenv("CO_CAPSULE_ID"),
                    attach_params=attach_raw,
                )
                print(
                    f"✓ Attached raw session asset '{raw_asset.name}' "
                    f"for base {session_base}"
                )
        except Exception as e:
            print(f"⚠️ Error attaching raw session for base {session_base}: {e}")

    # --- Existing videoprocessed logic (unchanged) ---
    try:
        params = DataAssetSearchParams(
            offset=0, limit=200, sort_order="desc", sort_field="name",
            archived=False, favorite=False, query=f"name:{session_base}"
        )
        res = client.data_assets.search_data_assets(params)
        # keep only assets that contain '_videoprocessed_' AND were processed on allowed days
        vp_assets = []
        for a in res.results:
            if "_videoprocessed_" not in a.name:
                continue
            day, dt = _parse_vp_dt(a.name)
            if day in _allowed_days and dt is not None:
                vp_assets.append((a, dt))
        if not vp_assets:
            print(f"⚠️ No match found for {session}")
            continue

        # pick most recent by parsed timestamp (second date after 'videoprocessed')
        latest_asset, _latest_dt = max(vp_assets, key=lambda x: x[1])

        # Attach using the asset's own mount name (as requested)
        attach = [DataAssetAttachParams(id=latest_asset.id, mount=latest_asset.mount)]
        _ = client.capsules.attach_data_assets(
            capsule_id=os.getenv("CO_CAPSULE_ID"),
            attach_params=attach,
        )

        # The folder for this session is the mounted path under data_root
        folder = data_root / latest_asset.mount

    except Exception as e:
        print(f"⚠️ CO API error for {session}: {e}")
        continue

    # --- look for predictions.csv under folder ---
    matches = list((folder / "pred_outputs" / "video_preds").glob("*predictions.csv"))
    if not matches:
        print(f"⚠️ No predictions.csv files found for {session}")
    else:
        # If multiple matches, pick the most recent (sorted by name)  [unchanged]
        csv_path = sorted(matches)[-1]
        if csv_path.exists():
            pred_csv_list.append(str(csv_path))
        else:
            print(f"⚠️ Predictions CSV missing for {session}")

# Print and save results (unchanged)
print("pred_csv_list = [")
for path in pred_csv_list:
    print(f'    "{path}",')
print("]")

save_path = Path("/root/capsule/scratch/pred_csv_list_20250113.json")
save_path.parent.mkdir(parents=True, exist_ok=True)

with open(save_path, "w") as f:
    json.dump(pred_csv_list, f, indent=2)

print(f"Saved {len(pred_csv_list)} paths to {save_path}")


⚠️ No match found for behavior_717121_2024-06-15_10-00-58
✓ Attached raw session asset 'behavior_781166_2025-05-13_14-04-22' for base behavior_781166_2025-05-13
⚠️ No predictions.csv files found for behavior_781166_2025-05-13_14-04-27
✓ Attached raw session asset 'behavior_781166_2025-05-14_14-18-24' for base behavior_781166_2025-05-14
⚠️ No predictions.csv files found for behavior_781166_2025-05-14_14-18-28
⚠️ No predictions.csv files found for behavior_781166_2025-05-15_14-20-51
✓ Attached raw session asset 'behavior_784806_2025-06-17_14-59-23' for base behavior_784806_2025-06-17
✓ Attached raw session asset 'behavior_784806_2025-06-18_13-39-50' for base behavior_784806_2025-06-18
✓ Attached raw session asset 'behavior_784806_2025-06-20_13-39-16' for base behavior_784806_2025-06-20
✓ Attached raw session asset 'behavior_791691_2025-06-24_13-21-26' for base behavior_791691_2025-06-24
✓ Attached raw session asset 'behavior_791691_2025-06-25_14-05-48' for base behavior_791691_2025-06-25